In [0]:
# 99_move_new_patient_encounter_files_to_raw.ipynb
# Move newly uploaded Patients and Encounters test files into their raw folders for Auto Loader.
# Files already present in the destination folder will be skipped (idempotent).

UPLOADS_DIR = "dbfs:/FileStore/tables/"
RAW_PATIENTS_DIR = "dbfs:/kardia/raw/patients/"
RAW_ENCOUNTERS_DIR  = "dbfs:/kardia/raw/encounters/"

# Mapping from filename prefix to raw destination folder
PREFIX_MAP = {
    "patients_part_": RAW_PATIENTS_DIR,
    "encounters_part_": RAW_ENCOUNTERS_DIR,
}

def _exists(dir_path: str, fname: str) -> bool:
    """Check if a file with the given name already exists in the destination directory."""
    return fname in [obj.name for obj in dbutils.fs.ls(dir_path)]

# Move each matching uploaded file, skipping if it already exists
for obj in dbutils.fs.ls(UPLOADS_DIR):
    dest_dir = next((path for prefix, path in PREFIX_MAP.items()
                     if obj.name.startswith(prefix)), None)

    if not dest_dir:
        continue  # Doesn't match expected prefixes

    if _exists(dest_dir, obj.name):
        print(f"Skipped (already exists): {obj.name}")
    else:
        dbutils.fs.cp(obj.path, dest_dir + obj.name)
        print(f"Moved: {obj.name} to {dest_dir}")
